In [126]:
import pandas as pd
import numpy as np
import math

In [127]:
def get_binary_address(given):
    global address_size
    
    format_str = "{0:0"+str(address_size)+"b}"
    res = format_str.format(int(given,16))
    return res

def get_block_offset(words):
    bo_size = int(math.log(words,2))
    #print(bo_size," bits")
    return bo_size

def get_index(words,word_size,cache_size):
    nr_lines = cache_size/(words*word_size)
    index_size = int(math.log(nr_lines,2))
    #print(index_size, "bits")
    return index_size

def get_tag(addr_sz,bo_sz,index_sz,byte_offset_sz):
    tag = addr_sz - index_sz - bo_sz
    #print(tag," bits")
    return tag

def calc_tag_index(index_len,bo_len,addr):
       
    tag_len = len(addr) - index_len - bo_len
    tag = addr[:tag_len]
    index = addr[tag_len:tag_len+index_len]
    #print("\nindex:",index,"\ntag:",tag)
    if(not index):
        return 0, int(tag,2)
    else:
        return int(index, 2), int(tag,2)

def get_cache_entry(given_addr):
    #address tag and index conversion
    #print("\n\naddress:",given_addr)
    binary_addr = get_binary_address(given_addr)

    #binary address
    #print("full bit address:\n", binary_addr + "00")
    #print("word address:\n",binary_addr)

    #tag and index
    index_val,tag_val = calc_tag_index(index,block_offset,binary_addr)
    #print("tag:",tag_val,"\nindex:",index_val)
    
    cache_entry = {"given_addr":given_addr,"w_addr":binary_addr,"tag":tag_val,"index":index_val}
    #print(cache_entry)
    
    return cache_entry


## Direct address format

In [128]:
#cache address format params

#address params (in bits)
byte_offset = 2
address_size = 32 - byte_offset

#word
word_size_bits = 32
word_size = word_size_bits/8

#cache
words_per_line = 2
cache_size = 8 * word_size


block_offset = get_block_offset(words_per_line)
index = get_index(words_per_line,word_size,cache_size)
tag = get_tag(address_size, block_offset, index, byte_offset)


#address format output
print("address format:\ntag ",tag,"\t index ",index,"\t block_offset",block_offset)

address format:
tag  27 	 index  2 	 block_offset 1


## Set associative cache

In [138]:
#cache address format params

#address params (in bits)
byte_offset = 2
address_size = 14 - byte_offset

#word
word_size_bits = 32 # 4 B
word_size = word_size_bits/8

#cache
words_per_line = 2
cache_size = 16 * word_size
associativity = 2

nr_index = cache_size / (words_per_line * associativity * word_size)


block_offset = get_block_offset(words_per_line)
index = int(math.log(nr_index,2))
tag = get_tag(address_size, block_offset, index, byte_offset)


#address format output
print("address format:\ntag ",tag,"\t index ",index,"\t block_offset",block_offset)
print("sets:",nr_index)
print("number of entries in cache:",cache_size / (words_per_line * word_size))

address format:
tag  9 	 index  2 	 block_offset 1
sets: 4.0
number of entries in cache: 8.0


## Fully associative

In [130]:
#cache address format params

#address params (in bits)
byte_offset = 2
address_size = 32 - byte_offset

#word
word_size_bits = 32
word_size = word_size_bits/8

#cache
words_per_line = 2
cache_size = 8 * word_size

block_offset = get_block_offset(words_per_line)
index = 0
tag = get_tag(address_size, block_offset, index, byte_offset)


#address format output
print("address format:\ntag ",tag,"\t index ",index,"\t block_offset",block_offset)

address format:
tag  29 	 index  0 	 block_offset 1


## Mapped cache address order from boot

In [131]:


addr_seq = '0x3, 0xb4, 0x2b, 0x02, 0xbf, 0x58, 0xbe, 0x0e, 0xb5, 0x2c, 0xba, 0xfd'
addr_seq = addr_seq.split(", ")


cache = pd.DataFrame(columns = ["given_addr","w_addr","tag","index"])

for i in range(len(addr_seq)):
    
    #if decimal, uncomment
    #addr_seq[i] = hex(int(addr_seq[i]))
    
    
    cache_entry = get_cache_entry(addr_seq[i])
    cache_hit = cache[cache['index'] == cache_entry['index']]
    if(len(cache_hit)>0):
        cache_hit = cache_hit[cache_hit['tag'] == cache_entry['tag']]
        if(len(cache_hit)>0):
            cache_entry["H/M"] = 1
            #cache_entry["replace"] = 0
        else:
            cache_entry["H/M"] = 0
            #cache_entry["replace"] = 1
    else:
        cache_entry["H/M"] = 0
        #cache_entry["replace"] = 0
        
    
    cache = cache.append(cache_entry,ignore_index=True)

    

# hits = cache['H/M'].sum()
# misses = len(cache) - hits
# hit_rate = hits / len(cache)
# #replaces = cache['replace'].sum()

# print("hits / misses / hit_rate:",hits,"/",misses,"/",hit_rate)
print("HIT MISS CANNOT BE TRUSTED WITHOUT VALID BIT")
print("CONFIRM HIT MISS MANUALLY")
cache

HIT MISS CANNOT BE TRUSTED WITHOUT VALID BIT
CONFIRM HIT MISS MANUALLY


,given_addr,w_addr,tag,index,H/M
0,0x3,000000000000000000000000000011,1,0,0.0
1,0xb4,000000000000000000000010110100,90,0,0.0
2,0x2b,000000000000000000000000101011,21,0,0.0
3,0x02,000000000000000000000000000010,1,0,1.0
4,0xbf,000000000000000000000010111111,95,0,0.0
5,0x58,000000000000000000000001011000,44,0,0.0
6,0xbe,000000000000000000000010111110,95,0,1.0
7,0x0e,000000000000000000000000001110,7,0,0.0
8,0xb5,000000000000000000000010110101,90,0,1.0
9,0x2c,000000000000000000000000101100,22,0,0.0


## Cache specs

In [132]:
tag = 22
index = 5
offset = 5

word_size = 4
nr_blocks_line = 2**offset
nr_cache_entries = 2**index
nr_blocks_cache = nr_blocks_line * nr_cache_entries

valid_bit = 1

total_bits_per_entry_per_line = valid_bit + tag + nr_blocks_line*8 #converting to bits

print("no. of blocks in one line:",nr_blocks_line,"B or",nr_blocks_line/word_size,"words","\nno. of entries:",nr_cache_entries)
print("no. of blocks in cache:",nr_blocks_cache,"B or",nr_blocks_cache/word_size, "words")

print("\ntotal bits per entry per line:", total_bits_per_entry_per_line)
print("total *data* bits per line:", nr_blocks_line*8)




no. of blocks in one line: 32 B or 8.0 words 
no. of entries: 32
no. of blocks in cache: 1024 B or 256.0 words

total bits per entry per line: 279
total *data* bits per line: 256
